<a href="https://colab.research.google.com/github/Rosalesmariana/Simulacion-I-/blob/main/Inventario_5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Inventario**

El sistema de inventario es un método de control de almacén, que permite asegurar el aprovisionamiento de mercancías y reducir los costos por pedido y por material faltante.

Para el desarrollo de este problema, se tomó como referencia el ejemplo 5.5 del libro de Coss-Bu

Dónde como resultado final, se obtiene una tabla que muestra los datos de nuestro inventario

Los parámetros para la obtención de estos, fueron los factores estacionales, el valor de x segun el número aleatorio asignado, que para mejor manejo y menor cantidad de lineas de código se trataron mediante un arreglo, se tomó a $q=200$ y a $R=100$ donde $q$ es el tamaño de la orden y $R$ el nivel de reorden, asi mismo se tomo como inventario inicial $150pz$

Una vez considerado esto, implementamos el siguiente código


In [22]:
import random
import pandas as pd

# Factores estacionales
factores_estacionales = [1.20, 1.00, 0.90, 0.80, 0.80, 0.70, 0.80, 0.90, 1.00, 1.20, 1.30, 1.40]

# Rango de valores de x basado en R
rangos_x = [
    (0.000, 0.010, 35), (0.010, 0.025, 36), (0.025, 0.045, 37), (0.045, 0.065, 38),
    (0.065, 0.087, 39), (0.087, 0.110, 40), (0.110, 0.135, 41), (0.135, 0.162, 42),
    (0.162, 0.190, 43), (0.190, 0.219, 44), (0.219, 0.254, 45), (0.254, 0.299, 46),
    (0.299, 0.359, 47), (0.359, 0.424, 48), (0.424, 0.494, 49), (0.494, 0.574, 50),
    (0.574, 0.649, 51), (0.649, 0.719, 52), (0.719, 0.784, 53), (0.784, 0.844, 54),
    (0.844, 0.894, 55), (0.894, 0.934, 56), (0.934, 0.964, 57), (0.964, 0.980, 58),
    (0.980, 0.995, 59), (0.995, 1.000, 60)
]

# Parámetros de la simulación
q = 200
R = 100
inventario_inicial = 150
num_meses = 12

# Función para obtener el valor de x basado en un número aleatorio R
def obtener_x(r):
    for (min_r, max_r, x) in rangos_x:
        if min_r <= r < max_r:
            return x
    return None

# Inicialización de las listas para cada columna de nuestra tabla de resultados
meses = list(range(1, num_meses + 1))
inventario_iniciales = [inventario_inicial]
numeros_aleatorios = []
demandas_ajustadas = []
inventario_finales = []
faltantes = []
numeros_ordenes = []
inventario_mensual = []

for mes in meses:
    numero_aleatorio = random.uniform(0, 1)  #Segun el numero aleatorio asignado
    x = obtener_x(numero_aleatorio) #Obtenemos el valor de X
    # Y podemos realizar las operaciones correspondientes
    demanda_ajustada = x * factores_estacionales[mes - 1]
    inventario_inicial = inventario_iniciales[-1]
    inventario_final = inventario_inicial - demanda_ajustada

    if inventario_final < 0: # Comprobamos si se tienen piezas faltantes
        faltante = abs(inventario_final)
        inventario_final = 0
    else:
        faltante = 0
    # Calculamos nuestro inventario mensual promedio
    inventario_promedio = (inventario_inicial + inventario_final) / 2
    numero_orden = 0

    if inventario_final < R: # Comprobamos si tenemos una cantidad por debajo del nivel de reorden
        numero_orden = q #Mostramos de manera explicita la cantidad, esto para diferenciar con el caso siguente

    # Agregar los valores calculados a las listas correspondientes
    inventario_iniciales.append(inventario_final)
    numeros_aleatorios.append(numero_aleatorio)
    demandas_ajustadas.append(demanda_ajustada)
    inventario_finales.append(inventario_final)
    faltantes.append(faltante)
    numeros_ordenes.append(numero_orden)
    inventario_mensual.append(inventario_promedio)

    # Actualizar el inventario inicial del siguiente mes si hay una orden
    if numero_orden > 0:
        inventario_iniciales[-1] += numero_orden

# Crear una tabla con los resultados
df = pd.DataFrame({
    'Mes': meses,
    'Inventario Inicial': inventario_iniciales[:-1],
    'Número Aleatorio': numeros_aleatorios,
    'Demanda Ajustada': demandas_ajustadas,
    'Inventario Final': inventario_finales,
    'Faltante': faltantes,
    'Número de Orden': numeros_ordenes,
    'Inventario Mensual': inventario_mensual
})

print(df)


    Mes  Inventario Inicial  Número Aleatorio  Demanda Ajustada  \
0     1               150.0          0.125369              49.2   
1     2               100.8          0.820104              54.0   
2     3               246.8          0.293005              41.4   
3     4               205.4          0.323574              37.6   
4     5               167.8          0.844432              44.0   
5     6               123.8          0.624320              35.7   
6     7               288.1          0.867185              44.0   
7     8               244.1          0.231941              40.5   
8     9               203.6          0.109690              40.0   
9    10               163.6          0.530645              60.0   
10   11               103.6          0.560995              65.0   
11   12               238.6          0.132026              57.4   

    Inventario Final  Faltante  Número de Orden  Inventario Mensual  
0              100.8         0                0           

**Algoritmo para encontrar la solución óptima**

Para fines de esta tarea, y siguiendo la sugerencia de la bibliografía referenciada, haremos uso del algoritmo de Hooke y Jeeaves

*Algoritmo de* **Hooke y Jeeaves**


El metodo de patrones de busqueda de Hooke-Jeeves crea un conjunto
de direcciones de búsqueda de manera iterativa. Este método fue propuesto en
1966 y fue uno de los primeros algoritmos en incorporar la historia previa de una secuencia de iteraciones en la generacion de una nueva direccion de busqueda.
El algoritmo combina movimientos exploratorios y movimientos de patrones. Es un método de optimización iterativo que no requiere la evaluación de derivadas, lo que lo hace adecuado para problemas en los que las derivadas no están disponibles o son difíciles de calcular.

En este caso, el algoritmo se utiliza para encontrar los valores óptimos de $q$ y $R$ que minimicen el costo total de inventario



Para la implementación de nuestro nuevo código, tendremos lo siguiente:

Punto de partida: Se elige un punto de inicio en el espacio de soluciones, en este caso, un par inicial $(q,R)$

Exploracion local: A partir del punto actual, el algoritmo realiza una exploración local en diferentes direcciones para buscar una mejora en la función objetivo que en este caso seria nuestro costo total

Bsqueda de patrón: Si se encuentra una mejora durante la exploracin local, el algoritmo realiza un movimiento más grande en esa dirección, siguiendo un patrón, si la búsqueda de patrón mejora la solucion, el nuevo punto se convierte en el punto actual

Iteraciones: El proceso de exploración local y búsqueda de patrón se repite iterativamente hasta que no se encuentren mejoras significativas

Siguiendo el codigo anterior, únicamnete añadimos la parte donde llevamos a cabo el algoritmo de Hooke y Jeeves

In [24]:
import random
import pandas as pd

# Factores estacionales
factores_estacionales = [1.20, 1.00, 0.90, 0.80, 0.80, 0.70, 0.80, 0.90, 1.00, 1.20, 1.30, 1.40]

# Rango de valores de x basado en R
rangos_x = [
    (0.000, 0.010, 35), (0.010, 0.025, 36), (0.025, 0.045, 37), (0.045, 0.065, 38),
    (0.065, 0.087, 39), (0.087, 0.110, 40), (0.110, 0.135, 41), (0.135, 0.162, 42),
    (0.162, 0.190, 43), (0.190, 0.219, 44), (0.219, 0.254, 45), (0.254, 0.299, 46),
    (0.299, 0.359, 47), (0.359, 0.424, 48), (0.424, 0.494, 49), (0.494, 0.574, 50),
    (0.574, 0.649, 51), (0.649, 0.719, 52), (0.719, 0.784, 53), (0.784, 0.844, 54),
    (0.844, 0.894, 55), (0.894, 0.934, 56), (0.934, 0.964, 57), (0.964, 0.980, 58),
    (0.980, 0.995, 59), (0.995, 1.000, 60)
]

# Parámetros de la simulación
inventario_inicial_global = 150
num_meses = 12

# Costos
C_p = 100  # Costo por pedido
C_m = 20 / 12  # Costo de mantenimiento por unidad por mes (20 por unidad por año, dividido entre 12 meses)
C_f = 50   # Costo por unidad faltante

# Función para obtener el valor de x basado en un número aleatorio R
def obtener_x(r):
    for (min_r, max_r, x) in rangos_x:
        if min_r <= r < max_r:
            return x
    return None

# Creamos una funcion para el desarrollo del inventario
def simulacion_inventario(q, R, inventario_inicial):
    inventario_iniciales = [inventario_inicial] # Inicialización de las listas para cada columna de nuestra tabla de resultados
    numeros_aleatorios = []
    demandas_ajustadas = []
    inventario_finales = []
    faltantes = []
    numeros_ordenes = []
    inventario_mensual = []
    costo_total = 0

    for mes in range(1, num_meses + 1):
        numero_aleatorio = random.uniform(0, 1) #Segun el numero aleatorio asignado
        x = obtener_x(numero_aleatorio) #Obtenemos el valor de X
        # Y podemos realizar las operaciones correnpondientes
        demanda_ajustada = x * factores_estacionales[mes - 1]
        inventario_inicial = inventario_iniciales[-1]
        inventario_final = inventario_inicial - demanda_ajustada

        if inventario_final < 0: # Comprobamos si se tienen piezas faltantes
            faltante = abs(inventario_final)
            inventario_final = 0
        else:
            faltante = 0
        # Calculamos nuestro inventario mensual promedio
        inventario_promedio = (inventario_inicial + inventario_final) / 2
        numero_orden = 0

        if inventario_final < R: # Comprobamos si tenemos una cantidad por debajo del nivel de reorden
        # A diferencia del codigo anterior, se moestrara el valor optimo de q
            numero_orden = q # Que se obtien una vez aplicado el algoritmo de Hooke y Jeeaves

        # Cálculo de costos
        costo_pedido = C_p if numero_orden > 0 else 0
        costo_mantenimiento = inventario_promedio * C_m
        costo_faltante = faltante * C_f
        costo_total += costo_pedido + costo_mantenimiento + costo_faltante

        # Agregar los valores calculados a las listas correspondientes
        inventario_iniciales.append(inventario_final)
        numeros_aleatorios.append(numero_aleatorio)
        demandas_ajustadas.append(demanda_ajustada)
        inventario_finales.append(inventario_final)
        faltantes.append(faltante)
        numeros_ordenes.append(numero_orden)
        inventario_mensual.append(inventario_promedio)

        # Actualizar el inventario inicial del siguiente mes si hay una orden
        if numero_orden > 0:
            inventario_iniciales[-1] += numero_orden

    # Crear una tabla con los resultados
    df = pd.DataFrame({
        'Mes': list(range(1, num_meses + 1)),
        'Inventario Inicial': inventario_iniciales[:-1],
        'Número Aleatorio': numeros_aleatorios,
        'Demanda Ajustada': demandas_ajustadas,
        'Inventario Final': inventario_finales,
        'Faltante': faltantes,
        'Número de Orden': numeros_ordenes,
        'Inventario Mensual': inventario_mensual
    })

    return costo_total, df

"""Algoritmo de Hooke y Jeeaves"""

# Función de búsqueda de patrón
def busqueda_patron(base_q, base_R, delta_q, delta_R):
    mejor_q = base_q
    mejor_R = base_R
    mejor_costo, _ = simulacion_inventario(mejor_q, mejor_R, inventario_inicial_global)

    # Probar cada dirección
    for dq, dR in [(delta_q, 0), (-delta_q, 0), (0, delta_R), (0, -delta_R)]:
        nuevo_q = mejor_q + dq
        nuevo_R = mejor_R + dR

        if nuevo_q <= 0 or nuevo_R <= 0:
            continue

        costo_actual, _ = simulacion_inventario(nuevo_q, nuevo_R, inventario_inicial_global)
        if costo_actual < mejor_costo:
            mejor_q = nuevo_q
            mejor_R = nuevo_R
            mejor_costo = costo_actual

    return mejor_q, mejor_R, mejor_costo

# Asignamos parámetros iniciales, para que a partir de estos se lleve a cabo el algoritmo
q_inicial = 200
R_inicial = 100
delta_q = 20
delta_R = 20

# Iteraciones del algoritmo
max_iteraciones = 100
q_actual = q_inicial
R_actual = R_inicial

for _ in range(max_iteraciones):
    nuevo_q, nuevo_R, nuevo_costo = busqueda_patron(q_actual, R_actual, delta_q, delta_R)

    if (nuevo_q, nuevo_R) == (q_actual, R_actual):
        if delta_q > 1 and delta_R > 1:
            delta_q //= 2
            delta_R //= 2
        else:
            break
    else:
        q_actual = nuevo_q
        R_actual = nuevo_R

# Resultados finales
costo_total_final, df_resultado_final = simulacion_inventario(q_actual, R_actual, inventario_inicial_global)

print(f" q_optimo: {q_actual}, R_optimo: {R_actual}, Costo Total: {costo_total_final}")
print(df_resultado_final)


 q_optimo: 162, R_optimo: 40, Costo Total: 3135.3333333333358
    Mes  Inventario Inicial  Número Aleatorio  Demanda Ajustada  \
0     1               150.0          0.346911              56.4   
1     2                93.6          0.753501              53.0   
2     3                40.6          0.936397              51.3   
3     4               162.0          0.965831              46.4   
4     5               115.6          0.164145              34.4   
5     6                81.2          0.108675              28.0   
6     7                53.2          0.012069              28.8   
7     8               186.4          0.141247              37.8   
8     9               148.6          0.009484              35.0   
9    10               113.6          0.644629              61.2   
10   11                52.4          0.376792              62.4   
11   12               162.0          0.998698              84.0   

    Inventario Final  Faltante  Número de Orden  Inventario Mensua